In [4]:
print(os.listdir())

['.ipynb_checkpoints', 'dukest1resurvey2009.3d', 'dukest1resurvey2009.gltf', 'dukest1resurvey2009.gltf.import', 'dukest1resurvey2009.js', 'dukest1resurvey2009.tscn', 'DukeStResurvey-drawnup-p3.jpg', 'DukeStResurvey-drawnup-p3.jpg.import', 'parse3ddmp.py', 'parse3D_work.ipynb', '__pycache__']


In [7]:
from parse3ddmp import DMP3d
import os

dump3dexe = '"C:\\Program Files (x86)\\Survex\\dump3d.exe"'
fname = "dukest1resurvey2009.3d"
dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)

"""
    self.nodepmap = { }   # { p: ([names], [flagsconsolidated] }  
    self.nodes = [ ]      # [ (p, name, [flags]) ]   (not strictly necessary, except to get entrance name before flagconsolidation)
    self.lines = [ ]      # [ (prevp, p, name, style, [flags], date) ]
    self.xsects = [ [ ] ] # [ [ (name, lrud) ] ]
    self.nmapnodes = { }  # { name: p } (derived from nodepmap, needed for xsects)
""";

In [66]:
from parse3ddmp import P3
import json

leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
xsects = [xsect  for xsect in dmp3d.xsects  if len(xsect) >= 2]

assert not dmp3d.cs, "We need to convert from absolute coords in this case"

# points as triples
stationpoints = set()
stationpoints.update(line[0]  for line in leglines)
stationpoints.update(line[1]  for line in leglines)
stationpoints = list(stationpoints)
stationpoints.sort(key=lambda X: (X[2], X[0], X[1]))

# centralize the sketch
def rangerat(ps, i, maxfac):
    return (1-maxfac)*min(p[i]  for p in ps) + maxfac*max(p[i]  for p in ps)
svxp0 = P3(rangerat(stationpoints, 0, 0.5), rangerat(stationpoints, 1, 0.5), rangerat(stationpoints, 2, 1.0))

stationpointsdict = dict((p, i)  for i, p in enumerate(stationpoints))
stationpointscoords = sum((tuple(stationpoint - svxp0)  for stationpoint in stationpoints), ())
stationpointsnames = [ dmp3d.nodepmap[stationpoint][0][0]  for stationpoint in stationpoints ]
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsstyles = [ legline[3]  for legline in leglines ]

xsectgps = [ ]
for xsectseq in xsects:
    xsectpoints = [dmp3d.nmapnodes[xs[0]]  for xs in xsectseq]
    xsectindexes = [ stationpointsdict[xsectpoint]  for xsectpoint in xsectpoints ]
    xsectrightvecs = [ ]
    for i in range(len(xsectpoints)):
        pback = xsectpoints[max(0, i-1)]
        p = xsectpoints[i]
        pfore = xsectpoints[min(len(xsectpoints)-1, i+1)]
        vback = P3.ZNorm(P3(p.x - pback.x, p.y - pback.y, 0.0))
        vfore = P3.ZNorm(P3(pfore.x - p.x, pfore.y - p.y, 0.0))
        xsectplanevec = P3.ZNorm(vback + vfore)
        xsectrightvecs.append(xsectplanevec[1])
        xsectrightvecs.append(-xsectplanevec[0])
    xsectrightvecs
    xsectlruds = sum((xs[1]  for xs in xsectseq), ())
    xsectlruds
    xsectgps.append({ "xsectindexes":xsectindexes, "xsectrightvecs":xsectrightvecs, "xsectlruds":xsectlruds })
xsectgps

jrec = { "stationpointscoords": stationpointscoords,
         "stationpointsnames":  stationpointsnames, 
         "legsconnections":     legsconnections,
         "legsstyles":          legsstyles, 
         "xsectgps":            xsectgps
       }

open("dukest1resurvey2009.json", "w").write(json.dumps(jrec))

3466

In [67]:
stationpoints

[P3(-11.7, 20.26, -0.82),
 P3(0.0, 0.0, 0.0),
 P3(10.04, -17.4, 0.0),
 P3(29.34, -12.94, 0.69),
 P3(45.02, -8.45, 0.69),
 P3(73.05, -21.52, 1.23),
 P3(92.46, -29.75, 1.23),
 P3(101.15, -48.4, 1.23),
 P3(130.17, -54.84, 1.38),
 P3(142.58, -66.01, 1.38),
 P3(111.11, -45.92, 1.59),
 P3(149.88, -79.74, 1.92),
 P3(163.77, -85.64, 2.19),
 P3(118.43, -45.67, 2.49),
 P3(118.43, -45.67, 3.21),
 P3(178.89, -93.4, 3.94),
 P3(171.55, -84.96, 4.13),
 P3(216.94, -111.11, 4.58),
 P3(191.37, -94.28, 5.25),
 P3(45.66, -11.42, 5.38),
 P3(209.13, -110.83, 5.68)]